In [ ]:
 !pip install naeval;
 !pip install nerus;
 !pip install corus;
 !pip install transformers;
 !pip install transformers[torch];
 !pip install seqeval;
 !pip install tokenizers;
 !pip install datasets;
 !pip install torch;
 !pip install accelerate -U;
 !pip install pymorphy3 faker bert-score;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e252990e865a20a6116476f5b09e4b87e16d92dd8254e1f2b386fbfefc17731b
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 120.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving annotations.bio to annotations.bio
Saving texts.txt to texts.txt


# импорт датасета



In [ ]:
!wget https://github.com/dialogue-evaluation/factRuEval-2016/archive/master.zip;
!unzip master.zip;
!rm master.zip;

--2026-01-16 17:47:08--  https://github.com/dialogue-evaluation/factRuEval-2016/archive/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/dialogue-evaluation/factRuEval-2016/zip/refs/heads/master [following]
--2026-01-16 17:47:09--  https://codeload.github.com/dialogue-evaluation/factRuEval-2016/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   2.58M  7.98MB/s    in 0.3s    

2026-01-16 17:47:10 (7.98 MB/s) - ‘master.zip’ saved [2708858]

Archive:  master.zip
dfe96237fce79fde13b2021d1938587e4ea3042e
   creating: factRuEval-2016-ma

#импорт зависимостей и настройка RuBERT

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer

#AutoTokenizer загрузка подходящего токенизатора по имени модели
#AutoModel базовая модель
#AutoModelForTokenClassification модель  для NER (классификация токенов)
#DataCollatorForTokenClassification коллатор, который паддит батчи и выравнивает метки токенов
#TrainingArguments, Trainer  высокоуровневый цикл обучения/валидации

from corus import load_factru
from datasets import Dataset, DatasetDict
from tqdm import tqdm


model_name = "DeepPavlov/rubert-base-cased"
max_tok_len = 512
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

#чтение датасета

In [ ]:
# [ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]
records = list(load_factru("factRuEval-2016-master"))

#формирование набора меток (BIO)

In [ ]:
types = [] #собираем все типы сущностей верхнего регистра. оставляем только уникальные
for rec in records:
    for obj in rec.objects:
        for span in obj.spans:
            types.append(span.type.upper())
types = sorted(set(types))
# === строим список всех меток BIO ===

# базовые типы из factRuEval, которые ты хочешь распознавать BERT'ом
base_types = ["LOC_NAME", "ORG_NAME", "SURNAME", "NAME", "PATRONYMIC"]

label_names = ["O"]  # токен не принадлежит никакой сущности

for t in base_types:
    label_names.append(f"B-{t}")  # начало сущности
    label_names.append(f"I-{t}")  # продолжение сущности

# добавляем метки из нашего дополнительного BIO-файла с ПДн
extra_tags = set()
with open("annotations.bio", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        _, tag = line.split("\t")
        if tag != "O":
            extra_tags.add(tag)

for tag in sorted(extra_tags):
    if tag not in label_names:
        label_names.append(tag)

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}
label2id

{'O': 0,
 'B-LOC_NAME': 1,
 'I-LOC_NAME': 2,
 'B-ORG_NAME': 3,
 'I-ORG_NAME': 4,
 'B-SURNAME': 5,
 'I-SURNAME': 6,
 'B-NAME': 7,
 'I-NAME': 8,
 'B-PATRONYMIC': 9,
 'I-PATRONYMIC': 10,
 'B-CADASTRAL': 11,
 'B-CARD': 12,
 'B-INN': 13,
 'B-MARRIAGE_CERT': 14,
 'B-PASSPORT': 15,
 'B-PHONE': 16,
 'B-SNILS': 17,
 'B-URL': 18,
 'I-CARD': 19,
 'I-MARRIAGE_CERT': 20,
 'I-PASSPORT': 21,
 'I-PHONE': 22,
 'I-SNILS': 23}

#разметка токенов BIO по спанам

In [ ]:
result = {
    'input_ids': [],
    'token_type_ids': [],
    'attention_mask': [],
    'labels': []
}
for rec in tqdm(records):
    labels = []
    tokenized_inputs = tokenizer(rec.text, truncation=True, max_length=max_tok_len, padding=True)
    enc = tokenized_inputs.encodings[0]
    offsets = enc.offsets
    word_ids = enc.word_ids
    stm = enc.special_tokens_mask
    temp_tags = []
    for i in range(len(offsets)):
        if word_ids[i] is None:
            labels.append(-100)
            temp_tags.append("O")
        elif i > 0 and word_ids[i-1] == word_ids[i]:
            labels.append(-100)
            offset = offsets[i]
            flg = False
            for obj in rec.objects:
                if flg:
                    break
                for span in obj.spans:
                    if span.start <= offset[0] < span.stop or span.start < offset[1] <= span.stop:
                        temp_tags.append(span.type)
                        flg = True
                        break
            if not flg:
                temp_tags.append("O")
        else:
            offset = offsets[i]
            flg = False
            for obj in rec.objects:
                if flg:
                    break
                for span in obj.spans:

                    if span.start <= offset[0] < span.stop or span.start < offset[1] <= span.stop:
                        if span.type == temp_tags[-1]:
                            prefix = "I-"
                        else:
                            prefix = "B-"
                        # labels.append(prefix + span.type.upper())
                        labels.append(label2id.get(prefix + span.type.upper(), 0))
                        temp_tags.append(span.type)
                        flg = True
                        break
            if not flg:
                labels.append(0)
                temp_tags.append("O")
    result['input_ids'].append(tokenized_inputs['input_ids'])
    result['token_type_ids'].append(tokenized_inputs['token_type_ids'])
    result['attention_mask'].append(tokenized_inputs['attention_mask'])
    result['labels'].append(labels)
    if len(labels) != len(offsets) or len(temp_tags) != len(offsets):
        break

100%|██████████| 254/254 [00:04<00:00, 56.58it/s]


#создание датасета

In [ ]:
#ds = Dataset.from_dict(result)
#ds

#разбиение датасета

In [ ]:
"""#создание датасетов"""

# 1) factRuEval -> датасет
ds_factru = Dataset.from_dict(result)

# 2) наш BIO-файл -> датасет с ПДн
extra_result = {
    "input_ids": [],
    "token_type_ids": [],
    "attention_mask": [],
    "labels": [],
}

tokens = []
tags = []

with open("annotations.bio", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()

        # пустая строка – конец одного текста
        if not line:
            if tokens:
                encoding = tokenizer(
                    tokens,
                    is_split_into_words=True,
                    truncation=True,
                    max_length=max_tok_len,
                )
                word_ids = encoding.word_ids()

                labels_ids = []
                prev_word_id = None
                for word_id in word_ids:
                    if word_id is None:
                        labels_ids.append(-100)
                    elif word_id != prev_word_id:
                        tag = tags[word_id]
                        labels_ids.append(label2id[tag])
                    else:
                        labels_ids.append(-100)
                    prev_word_id = word_id

                extra_result["input_ids"].append(encoding["input_ids"])
                extra_result["token_type_ids"].append(encoding["token_type_ids"])
                extra_result["attention_mask"].append(encoding["attention_mask"])
                extra_result["labels"].append(labels_ids)

                tokens, tags = [], []
            continue

        # строки-комментарии из BIO (например "# Text 1") пропускаем
        if line.startswith("#"):
            continue

        token, tag = line.split("\t")
        tokens.append(token)
        tags.append(tag)

# на случай, если файл не заканчивается пустой строкой
if tokens:
    encoding = tokenizer(
        tokens,
        is_split_into_words=True,
        truncation=True,
        max_length=max_tok_len,
    )
    word_ids = encoding.word_ids()
    labels_ids = []
    prev_word_id = None
    for word_id in word_ids:
        if word_id is None:
            labels_ids.append(-100)
        elif word_id != prev_word_id:
            tag = tags[word_id]
            labels_ids.append(label2id[tag])
        else:
            labels_ids.append(-100)
        prev_word_id = word_id

    extra_result["input_ids"].append(encoding["input_ids"])
    extra_result["token_type_ids"].append(encoding["token_type_ids"])
    extra_result["attention_mask"].append(encoding["attention_mask"])
    extra_result["labels"].append(labels_ids)

ds_extra = Dataset.from_dict(extra_result)

# 3) ОБЪЕДИНЯЕМ factRuEval и наш датасет ПДн и делим на train/valid/test
from datasets import concatenate_datasets, DatasetDict
import numpy as np

ds_full = concatenate_datasets([ds_factru, ds_extra])

gen = np.random.default_rng(42)

dsd = ds_full.train_test_split(test_size=0.2, generator=gen)
dsd_temp = dsd["test"].train_test_split(test_size=0.5, generator=gen)

dsd = DatasetDict({
    "train": dsd["train"],
    "test": dsd_temp["test"],
    "valid": dsd_temp["train"],
})

dsd


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1563
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 196
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 195
    })
})

#подготовка модели

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer) #при подаче данных в Trainer он динамически паддит последовательности до одинаковой длины
model = AutoModelForTokenClassification.from_pretrained(model_name, #загрузка предобученной модели
                                                        num_labels=len(label_names),
                                                        id2label=id2label,
                                                        label2id=label2id,
                                                        ignore_mismatched_sizes=True).to(device)
# model.config

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#параметры обучения

In [ ]:
training_args = TrainingArguments(
    output_dir="./train_result",
    eval_strategy="epoch", #оценка качества в конце каждой эпохи
    save_strategy="epoch",
    learning_rate=2e-5, #скорость обучения
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=16,  # надо подобрать оптимальное значение с т.з. качества
    logging_steps=50, #шаг логирования
    weight_decay=0.01, #
    load_best_model_at_end=True, #в конце подгрузить лучший чекпоинт по метрике
    report_to=None, #отправка логов в др сервисы
    save_safetensors=False
)

# оценка качества

In [ ]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)

    y_true = [[id2label[l] for l, p in zip(label, pred) if l != -100] for label, pred in zip(labels, predictions)] #Делаем списки меток для каждого примера: пропускаем позиции с -100
    y_pred = [[id2label[p] for l, p in zip(label, pred) if l != -100] for label, pred in zip(labels, predictions)] # и переводим числовые id в строковые BIO-метки через id2label

    res = classification_report(y_true=y_true, y_pred=y_pred, scheme=IOB2, output_dict=True)["weighted avg"] #считаем метрики по био тегам и берем средневзвешенные значения
    res.pop("support", None)
    return res

# запуск обучения

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"


training_args = TrainingArguments(
    output_dir="./train_result",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=16,
    logging_steps=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to=None,
    save_safetensors=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dsd["train"],
    eval_dataset=dsd["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-2974159885.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score
1,0.806400,0.071989,0.884590,0.899884,0.891245
2,0.044600,0.040718,0.957109,0.957755,0.957383
3,0.032000,0.038204,0.944842,0.965856,0.955049
4,0.018200,0.045278,0.959746,0.956019,0.957799
5,0.011700,0.038913,0.962666,0.968750,0.965685
6,0.011700,0.040163,0.960696,0.967014,0.963815
7,0.008200,0.040881,0.960479,0.970486,0.965417
8,0.006900,0.044603,0.954984,0.967593,0.961168
9,0.005300,0.043654,0.952606,0.969329,0.960637
10,0.004500,0.043670,0.958264,0.968171,0.963154


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score
1,0.806400,0.071989,0.884590,0.899884,0.891245
2,0.044600,0.040718,0.957109,0.957755,0.957383
3,0.032000,0.038204,0.944842,0.965856,0.955049
4,0.018200,0.045278,0.959746,0.956019,0.957799
5,0.011700,0.038913,0.962666,0.968750,0.965685
6,0.011700,0.040163,0.960696,0.967014,0.963815
7,0.008200,0.040881,0.960479,0.970486,0.965417
8,0.006900,0.044603,0.954984,0.967593,0.961168
9,0.005300,0.043654,0.952606,0.969329,0.960637
10,0.004500,0.043670,0.958264,0.968171,0.963154


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=1568, training_loss=0.03874967486849435, metrics={'train_runtime': 2587.1034, 'train_samples_per_second': 9.666, 'train_steps_per_second': 0.606, 'total_flos': 4362654515547168.0, 'train_loss': 0.03874967486849435, 'epoch': 16.0})

# сохранение обученной модели на диск

In [ ]:
model_name = "./res_model_base"
trainer.save_model(model_name)

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForTokenClassification


model = AutoModelForTokenClassification.from_pretrained(model_name) #загружаем сохраненную модель
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

labels = []
predictions = []
for b in dsd["test"]:
    labels.append(b.pop("labels")) #забираем правильные метки
    batch = {k: torch.as_tensor([v]) for k, v in b.items()}
    with torch.no_grad():
        outputs = model(**batch)
    predictions.append(np.argmax(outputs.logits, axis=-1).numpy())

# объединение предиктов из разных батчей

In [ ]:
preds = []  # объединение предиктов из разных батчей
for i, ps in enumerate(predictions):
    for j, p in enumerate(ps):
        preds.append(p)

# labels = tokenized_dsd["test"]["labels"]  # подгрузка размеченных/истинных тегов
len(preds), len(labels) #проверяем, совпадает ли количество предсказаний и количество истинных разметок (должно быть одинаковым)

# Преобразование предсказаний в классы без B-/I-

In [ ]:
import re


def rm_bi(x): #удаляем префиксы
    return re.sub(r"^[BI]-", "", x).upper()

y_true = []
y_pred = []
for label, pred in zip(labels, predictions):
    for l, p in zip(label, pred[0]):
        if l != -100:
            y_true.append(rm_bi(id2label[l]))
            y_pred.append(rm_bi(id2label[p]))
lbls = list(set(y_true))
lbls.sort()
len(y_true), len(y_pred) #проверка количества токенов

# метрики по классам: precision/recall/F1

In [ ]:
from sklearn.metrics import classification_report

rep = classification_report(y_true=y_true, y_pred=y_pred, labels=lbls, output_dict=False, digits=4, zero_division=0)
print(rep)

               precision    recall  f1-score   support

    CADASTRAL     1.0000    1.0000    1.0000        70
         CARD     1.0000    1.0000    1.0000       276
          INN     1.0000    1.0000    1.0000        80
     LOC_NAME     0.9044    0.8849    0.8945       139
MARRIAGE_CERT     1.0000    1.0000    1.0000        90
         NAME     0.9602    1.0000    0.9797       241
            O     0.9973    0.9941    0.9957      7482
     ORG_NAME     0.7584    0.8599    0.8060       157
     PASSPORT     1.0000    1.0000    1.0000       160
   PATRONYMIC     0.0000    0.0000    0.0000        11
        PHONE     1.0000    1.0000    1.0000       486
        SNILS     1.0000    1.0000    1.0000       444
      SURNAME     0.9741    1.0000    0.9869       263
          URL     1.0000    1.0000    1.0000        66

     accuracy                         0.9907      9965
    macro avg     0.8996    0.9099    0.9045      9965
 weighted avg     0.9901    0.9907    0.9903      9965



# извлечение сущностей из текста

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import numpy as np
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "./res_model_base"
model = AutoModelForTokenClassification.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

id2label = model.config.id2label #перевод индекс в текстовую метку

def predict_entities(text):
    tokens = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True).to(device) #токенизируем текст, для каждого токена находим границы в строке
    offset_mapping = tokens.pop("offset_mapping")  # убираем, чтобы не передавалось в модель

    outputs = model(**tokens)
    preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()[0]
    offsets = offset_mapping[0].cpu().numpy()

    entities = []
    current_entity = None
    current_label = None

    for pred, (start, end) in zip(preds, offsets): #
        label = id2label.get(pred, "O")
        if label.startswith("B-"):
            if current_entity:
                entities.append([current_label, current_entity[0], current_entity[1]])
            current_label = label[2:]
            current_entity = [start, end]
        elif label.startswith("I-") and current_label == label[2:]:
            current_entity[1] = end
        else:
            if current_entity:
                entities.append([current_label, current_entity[0], current_entity[1]])
                current_entity = None
                current_label = None

    if current_entity:
        entities.append([current_label, current_entity[0], current_entity[1]])

    return entities

# правила распознавания ПДн

In [ ]:
import re

def find_email(text):
    return [[m.group(), m.start(), m.end()] for m in re.finditer(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', text)]

def find_inn(text):

    #Распознает ИНН физических лиц (12 цифр) и проверяет контрольные разряды.

    def check_inn(num):
        # Проверяем только 12-значный ИНН
        if len(num) != 12 or not num.isdigit():
            return False

        digits = list(map(int, num))
        #расчет 11-го знака
        coeff1 = [7, 2, 4, 10, 3, 5, 9, 4, 6, 8]
        n11 = sum(d * c for d, c in zip(digits[:10], coeff1)) % 11 % 10

        #расчет 12-го знака
        coeff2 = [3, 7, 2, 4, 10, 3, 5, 9, 4, 6, 8]
        n12 = sum(d * c for d, c in zip(digits[:11], coeff2)) % 11 % 10

        # Проверяем соответствие обоих контрольных разрядов
        return digits[10] == n11 and digits[11] == n12

    # Находим только 12-значные последовательности
    pattern = r'\b\d{12}\b'

    return [
        ["INN", m.start(), m.end()]
        for m in re.finditer(pattern, text)
        if check_inn(m.group())
    ]

def find_phone(text):
    pattern = r'(?<!\d)(?:\+7|8)[\s\-()]?\d{3}[\s\-()]?\d{3}[\s\-()]?\d{2}[\s\-()]?\d{2}(?!\d)'
    return [["PHONE", m.start(), m.end()] for m in re.finditer(pattern, text)]

def find_passport(text):
    # Серия: 4 цифры (первые две - код региона РФ из списка), затем 2 цифры года
    # Между серией и 6 значным номером должен быть хотя бы 1 разделитель: пробел, №, тире и т.п.
    pattern = re.compile(r'''
        (?<!\d) # не цифра слева
        (?: # серия паспорта:
           (?:01|03|04|05|07|08|10|11|12|14|15|17|18|19|20|21|22|24|25|26|27|28|29|32|33|34|35|36|37|38|40|41|42|43|
              44|45|46|47|48|49|50|51|52|53|54|55|56|57|58|59|60|61|62|63|64|65|66|67|68|69|70|71|72|73|74|75|76|77|
              78|79|80|81|82|83|84|85|86|87|88|89|90|91|92|93|94|95|96|97|98|99)
           \s?\d{2} # год (2 цифры), возможен 1 пробел внутри серии
        )
        (?:\s|[№Nn\-–—]){1,7} #разделитель между серией и номером
        \d{6} # номер (6 цифр)
        (?!\d) # не цифра справа
    ''', re.VERBOSE)
    return [["PASSPORT", m.start(), m.end()] for m in pattern.finditer(text)]

def find_card(text):
    def luhn_check(num):
        digits = [int(d) for d in num if d.isdigit()]
        checksum = 0
        parity = len(digits) % 2
        for i, d in enumerate(digits):
            if i % 2 == parity:
                d *= 2
                if d > 9:
                    d -= 9
            checksum += d
        return checksum % 10 == 0

    #16 цифр MIR (2[2-7] + ещё 14) или VISA (4 + ещё 15)
    pattern = r'\b(?:\d{4}[ -]?){3}\d{4}\b'

    return [
        ["CARD", m.start(), m.end()]
        for m in re.finditer(pattern, text)
        if luhn_check(m.group())
    ]

def find_email(text):
    # Локальная часть: без двойных точек, не начинается/заканчивается точкой
    # Доменная часть: без двойных точек, без точки в конце
    pattern = re.compile(
        r'\b'                              # граница слова
        r'[A-Za-z0-9](?:[A-Za-z0-9._%+-]*[A-Za-z0-9])?'  # локальная часть
        r'@'
        r'[A-Za-z0-9](?:[A-Za-z0-9-]*[A-Za-z0-9])?'      # доменное имя (до точки)
        r'(?:\.[A-Za-z]{2,})+'             # доменная зона (одна или больше)
        r'\b'
    )

    results = []
    for m in pattern.finditer(text):
        email = m.group()

        # апрещаем двойные точки в любом месте
        if ".." in email:
            continue

        results.append(["EMAIL", m.start(), m.end()])

    return results

def find_snils(text):
    """
    Поиск СНИЛС с проверкой контрольного числа.
    Допускаются:
    - одинаковые разделители между первыми тремя блоками
    - пробел перед последними двумя цифрами
    """

    pattern = r'(?<!\d)\d{3}([ ._-]?)\d{3}\1\d{3}[ ]?\d{2}(?!\d)'

    results = []

    for m in re.finditer(pattern, text):
        num = re.sub(r'\D', '', m.group())  # только цифры
        if len(num) != 11:
            continue

        digits = [int(d) for d in num]

        # вычисление контрольного числа
        weights = list(range(9, 0, -1))
        s = sum(d * w for d, w in zip(digits[:9], weights))
        r = s % 101
        ctrl_calc = 0 if r == 100 else r

        ctrl_orig = digits[9] * 10 + digits[10]

        if ctrl_calc == ctrl_orig:
            results.append(["SNILS", m.start(), m.end()])

    return results

def find_url(text):
    """
    http/https URL
    """
    pattern = re.compile(r'https?://[^\s)]+', re.IGNORECASE)
    return [["URL", m.start(), m.end()] for m in re.finditer(pattern, text)]

def find_marriage_cert(text):
    """
    Номер свидетельства о браке NN-NNNNNN.
    """
    pattern = r'\b\d{2}-\d{6}\b'
    return [["MARRIAGE_CERT", m.start(), m.end()] for m in re.finditer(pattern, text)]

def find_cadastral(text):
    """
    Кадастровый номер: XX:XX:XXXXXXXXX:XXXX ( 2:2:9:4 цифр).
    """
    pattern = r'\b\d{2}:\d{2}:\d{9}:\d{4}\b'
    return [["CADASTRAL", m.start(), m.end()] for m in re.finditer(pattern, text)]



# объединение BERT и регулярок

In [ ]:
def extract_pii(text):
    #BERT
    bert_entities = predict_entities(text)
    clean_bert_entities = []
    for label, start, end in bert_entities:
        label = str(label).upper()
        frag = text[start:end]

        #Фильтр ФИО: удаляем сущности с цифрами
        if label in {"NAME", "SURNAME", "PATRONYMIC"}:
            if any(ch.isdigit() for ch in frag):
                continue

        # ORG_NAME не является ПДн
        if label == "ORG_NAME":
            continue

        clean_bert_entities.append([label, int(start), int(end)])

    # REGEX
    regex_entities = []
    for func in [
        find_email,
        find_inn,
        find_phone,
        find_passport,
        find_card,
        find_snils,
        find_url,
        find_marriage_cert,
        find_cadastral,
    ]:
        regex_entities.extend(func(text))

    clean_regex_entities = []
    for e in regex_entities:
        label = e[0]
        if "@" in label:
            clean_regex_entities.append(["EMAIL", int(e[1]), int(e[2])])
        else:
            clean_regex_entities.append([str(label).upper(), int(e[1]), int(e[2])])

    #объединяем
    all_entities = clean_bert_entities + clean_regex_entities

    # приоритеты
    priority_labels = {"EMAIL", "INN", "CARD", "SNILS", "URL", "CADASTRAL", "MARRIAGE_CERT"}

    filtered = []
    for ent in all_entities:
        ent_label, ent_start, ent_end = ent

        inside_priority = False
        for other in all_entities:
            other_label, s, e = other
            if other is ent:
                continue

            if other_label in priority_labels:
                if ent_start >= s and ent_end <= e:
                    inside_priority = True
                    break

        if ent_label == "SNILS":
            filtered.append(ent)
            continue

        # удаляем вложенные не-приоритетные
        if inside_priority and ent_label not in priority_labels:
            continue

        filtered.append(ent)


    # ====== удаляем дубликаты по одинаковым границам ======
    unique = []
    for ent in filtered:
        replaced = False
        for k, added in enumerate(unique):
            # тот же span
            if ent[1] == added[1] and ent[2] == added[2]:
                # если у ent есть приоритет
                if ent[0] in priority_labels and added[0] not in priority_labels:
                    unique[k] = ent
                replaced = True
                break

        if not replaced:
            unique.append(ent)

    # ====== сортируем ======
    unique.sort(key=lambda x: x[1])

    return unique


def _digits_count(s: str) -> int:
    return sum(ch.isdigit() for ch in s)

def normalize_spans(text: str, spans):
    """
    spans: list of [LABEL, start, end]
    1) выкидываем явно "частичные" числовые ПДн
    2) убираем пересечения: оставляем более приоритетный/длинный
    """
    min_digits = {
        "INN": 10,          # 10 или 12
        "PASSPORT": 10,
        "SNILS": 11,
        "CARD": 16,
        "CADASTRAL": 17,
        "MARRIAGE_CERT": 8,
    }

    # 1) фильтр частичных
    filtered = []
    for label, s, e in spans:
        frag = text[s:e]
        if label in min_digits:
            if _digits_count(frag) < min_digits[label]:
                continue
        if label == "URL":
            if not frag.strip().lower().startswith(("http://", "https://")):
                continue
        filtered.append([label, s, e])

    # 2) разруливание пересечений (вес)
    prio = {
        "EMAIL": 100,
        "INN": 95,
        "SNILS": 95,
        "CARD": 95,
        "PASSPORT": 90,
        "CADASTRAL": 90,
        "MARRIAGE_CERT": 90,
        "URL": 80,
        # остальное ниже
        "SURNAME": 30,
        "NAME": 30,
        "PATRONYMIC": 30,
        "LOC_NAME": 10,
        "ORG_NAME": 0,
    }

    def key(ent):
        label, s, e = ent
        length = e - s
        return (s, -prio.get(label, 0), -length, label)

    filtered.sort(key=key)

    out = []
    for ent in filtered:
        label, s, e = ent
        if not out:
            out.append(ent); continue

        pl, ps, pe = out[-1]

        # нет пересечения
        if s >= pe:
            out.append(ent)
            continue

        # есть пересечение → выбираем лучшее
        cur_score = (prio.get(label, 0), e - s)
        prev_score = (prio.get(pl, 0), pe - ps)

        if cur_score > prev_score:
            out[-1] = ent
        # иначе оставляем предыдущий

    return out

# реализация модуля синонимического обезличивания

In [ ]:
%%writefile anon_utils.py
# -*- coding: utf-8 -*-

import re
from secrets import SystemRandom
import pymorphy3
from faker import Faker

rand = SystemRandom()
morph = pymorphy3.MorphAnalyzer()
fake = Faker("ru_RU")

def _preserve_capitalization(src: str, dst: str) -> str:
    if src.isupper():
        return dst.upper()
    if src.istitle() or (len(src) > 1 and src[0].isupper() and src[1:].islower()):
        return dst.capitalize()
    return dst

def _random_choice_name_by_gender(gender: str) -> str:
    if gender == "femn":
        return fake.first_name_female()
    return fake.first_name_male()

def _random_choice_surname_by_gender(gender: str) -> str:
    s = fake.last_name()
    if gender == "femn" and not s.endswith(("а", "ая", "на")):
        return s + "а"
    if gender == "masc" and s.endswith("а"):
        return s[:-1]
    return s

def _detect_gender_and_parse(token: str):
    p = morph.parse(token)[0]
    return p.tag.gender, p

def choose_name_strict(gender: str) -> str:
    """
    Гарантирует, что выбранное имя соответствует заданному роду
    """
    for _ in range(20):
        if gender == "femn":
            name = fake.first_name_female()
        else:
            name = fake.first_name_male()

        p = morph.parse(name)[0]
        if p.tag.gender == gender:
            return name

    return fake.first_name_female() if gender == "femn" else fake.first_name_male()


def _inflect_like(src: str, dst: str, src_parse) -> str:
    tags = set(src_parse.tag.grammemes)
    for pn in morph.parse(dst):
        if tags.issubset(set(pn.tag.grammemes)):
            return _preserve_capitalization(src, pn.word)
    for pn in morph.parse(dst):
        inf = pn.inflect(tags)
        if inf:
            return _preserve_capitalization(src, inf.word)
    return _preserve_capitalization(src, dst)

def detect_gender_from_context(text: str, start: int, end: int): #определение рода по контексту
    """
    Род по контексту (приоритет):
    1) отчество
    2) фамилия
    3) глагол прошедшего времени (морфологически проверяем, что это VERB past)
    """
    window_start = max(0, start - 80)
    window_end = min(len(text), end + 80)
    window = text[window_start:window_end]

    # 1) отчество
    if re.search(r"\b[А-ЯЁ][а-яё]+(овна|евна|ична|инична)\b", window):
        return "femn"
    if re.search(r"\b[А-ЯЁ][а-яё]+(ович|евич|ич|ыч)\b", window):
        return "masc"

    # 2) фамилия
    if re.search(r"\b[А-ЯЁа-яё-]+(ова|ева|ина|ская)\b", window, re.IGNORECASE):
        return "femn"
    if re.search(r"\b[А-ЯЁа-яё-]+(ов|ев|ин|ский)\b", window, re.IGNORECASE):
        return "masc"

    # 3) глагол прошедшего времени
    for m in re.finditer(r"\b[А-ЯЁа-яё-]{3,}\b", window):
        w = m.group(0)
        p = morph.parse(w)[0]
        grams = set(p.tag.grammemes)
        # VERB past имеет граммему 'past'; род даёт p.tag.gender
        if "VERB" in str(p.tag) and "past" in grams and p.tag.gender in ("masc", "femn"):
            return p.tag.gender

    return None

UNISEX = {"женя", "саша", "валя", "жаня", "шура"}  # можно расширить

def _pick_parse_by_gender(word: str, ctx_gender: str | None, tag_required: str):
    parses = morph.parse(word)
    parses = [p for p in parses if tag_required in p.tag] or parses

    if ctx_gender in ("masc", "femn"):
        by_gender = [p for p in parses if p.tag.gender == ctx_gender]
        if by_gender:
            # если есть вариант в именительном - берем его
            by_nom = [p for p in by_gender if "nomn" in p.tag.grammemes]
            return by_nom[0] if by_nom else by_gender[0]

    # иначе просто самый вероятный
    return parses[0]

def name_anon(name: str, text: str = None, start: int = None, end: int = None) -> str:
    if not isinstance(name, str) or not name or not name.isalpha():
        return "Некорректное имя"

    ctx_gender = None
    if text is not None and start is not None and end is not None:
        ctx_gender = detect_gender_from_context(text, start, end)

    # ключевой момент: выбираем разбор имени с учётом ctx_gender (исправляет “Александра”)
    parsed_old = _pick_parse_by_gender(name, ctx_gender, "Name")

    # род: для унисекс или если pymorphy не дал - берем из контекста
    gender = parsed_old.tag.gender
    if name.lower() in UNISEX or gender is None:
        if ctx_gender in ("masc", "femn"):
            gender = ctx_gender

    if gender not in ("masc", "femn"):
        gender = "masc"

    new_name = choose_name_strict(gender)
    parsed_new = morph.parse(new_name)[0]

    # склоняем только если род совпал
    if parsed_new.tag.gender == gender and parsed_old.tag.gender == gender:
        return _inflect_like(name, new_name, parsed_old)

    return _preserve_capitalization(name, parsed_new.normal_form)


def surname_anon(surname: str) -> str:
    if not isinstance(surname, str) or not surname or not surname.isalpha() or len(surname) < 2:
        return "Некорректная фамилия"
    parsed = morph.parse(surname)[0]
    if "Surn" not in parsed.tag:
        return "Некорректная фамилия"

    gender, parsed = _detect_gender_and_parse(surname)
    gender = "masc" if gender == "masc" else "femn"
    new_surname = _random_choice_surname_by_gender(gender)
    return _inflect_like(surname, new_surname, parsed)

def email_anon(email: str) -> str:
    if not isinstance(email, str) or not email or "@" not in email or " " in email:
        return "Некорректный email"
    local, domain = email.split("@", 1)
    if not local or not domain:
        return "Некорректный email"

    new_local = []
    for ch in local:
        if re.match(r"[A-Za-z0-9]", ch):
            new_local.append(rand.choice("abcdefghijklmnopqrstuvwxyz0123456789"))
        else:
            new_local.append(ch)
    return "".join(new_local) + "@" + domain

def _inn_checksum_10(d):
    w = [2,4,10,3,5,9,4,6,8]
    return (sum(x*y for x,y in zip(d[:9], w)) % 11) % 10

def _inn_checksum_11(d):
    w = [7,2,4,10,3,5,9,4,6,8]
    return (sum(x*y for x,y in zip(d[:10], w)) % 11) % 10

def _inn_checksum_12(d):
    w = [3,7,2,4,10,3,5,9,4,6,8]
    return (sum(x*y for x,y in zip(d[:11], w)) % 11) % 10

def inn_anon(inn: str) -> str:
    digits = [int(d) for d in re.findall(r"\d", inn)]
    if len(digits) not in (10, 12):
        return "Некорректный ИНН"

    first4 = digits[:4]

    if len(digits) == 10:
        # делаем 9 первых цифр и считаем 10-ю (контрольную)
        mid5 = [rand.randint(0, 9) for _ in range(5)]
        base9 = first4 + mid5  # 9 цифр
        ctrl = _inn_checksum_10(base9 + [0])  # можно и без +[0], но так ок
        return "".join(map(str, base9 + [ctrl]))

    # 12-значный: сначала 10 цифр, затем c11, затем c12
    mid6 = [rand.randint(0, 9) for _ in range(6)]
    base10 = first4 + mid6  # 10 цифр
    c11 = _inn_checksum_11(base10)
    base11 = base10 + [c11]
    c12 = _inn_checksum_12(base11)
    return "".join(map(str, base11 + [c12]))

def phone_anon(phone: str) -> str:
    if not isinstance(phone, str) or not phone or re.search(r"[A-Za-zА-Яа-я]", phone):
        return "Некорректный номер телефона"
    digits = re.findall(r"\d", phone)
    if len(digits) < 4:
        return "Некорректный номер телефона"

    keep = digits[:4]
    tail = [str(rand.randint(0,9)) for _ in range(len(digits)-4)]
    new_digits = keep + tail

    out, i = [], 0
    for ch in phone:
        if ch.isdigit():
            out.append(new_digits[i]); i += 1
        else:
            out.append(ch)
    return "".join(out)


def passport_anon(passport: str) -> str:
    # сохраняем формат, меняем цифры, оставляя первые 2 (регион) как косвенный признак
    digits = re.findall(r"\d", passport)
    if len(digits) < 10:
        return "Некорректный паспорт"
    keep = digits[:2]
    rest = [str(rand.randint(0,9)) for _ in range(len(digits)-2)]
    new_digits = keep + rest

    out, i = [], 0
    for ch in passport:
        if ch.isdigit():
            out.append(new_digits[i]); i += 1
        else:
            out.append(ch)
    return "".join(out)

def _luhn_make(digits15):
    s = 0
    parity = (len(digits15) + 1) % 2
    for i, d in enumerate(digits15):
        x = d
        if i % 2 == parity:
            x *= 2
            if x > 9:
                x -= 9
        s += x
    return (10 - (s % 10)) % 10

def card_anon(card: str) -> str:
    digits = [int(x) for x in re.findall(r"\d", card)]
    if len(digits) != 16:
        return "Некорректная карта"

    bin6 = digits[:6]
    last4 = digits[11:15]  # позиции 12..15 (перед контрольной)

    mid5 = [rand.randint(0, 9) for _ in range(5)]  # позиции 7..11

    first15 = bin6 + mid5 + last4  # 6 + 5 + 4 = 15
    check = _luhn_make(first15)    # считаем 16-ю

    new_digits = first15 + [check]  # 16 цифр

    # восстановим формат (пробелы/дефисы сохранятся)
    out, i = [], 0
    for ch in card:
        if ch.isdigit():
            out.append(str(new_digits[i])); i += 1
        else:
            out.append(ch)
    return "".join(out)

def snils_anon(snils: str) -> str:
    digits = [int(x) for x in re.findall(r"\d", snils)]
    if len(digits) != 11:
        return "Некорректный СНИЛС"

    first9 = [rand.randint(0,9) for _ in range(9)]
    s = sum(d*w for d,w in zip(first9, range(9,0,-1)))
    r = s % 101
    ctrl = 0 if r == 100 else r
    ctrl2 = [ctrl // 10, ctrl % 10]
    new_digits = first9 + ctrl2

    out, i = [], 0
    for ch in snils:
        if ch.isdigit():
            out.append(str(new_digits[i])); i += 1
        else:
            out.append(ch)
    return "".join(out)

def url_anon(url: str) -> str:
    # сохраняем схему и TLD, домен и путь “перемешиваем”
    m = re.match(r"^(https?://)([^/]+)(/.*)?$", url.strip())
    if not m:
        return "Некорректный URL"
    scheme, host, path = m.group(1), m.group(2), m.group(3) or ""
    parts = host.split(".")
    if len(parts) < 2:
        return "Некорректный URL"
    tld = parts[-1]
    new_domain = "".join(rand.choice("abcdefghijklmnopqrstuvwxyz") for _ in range(10))
    new_host = new_domain + "." + tld

    def repl_path(s):
        out = []
        for ch in s:
            if ch.isalnum():
                out.append(rand.choice("abcdefghijklmnopqrstuvwxyz0123456789"))
            else:
                out.append(ch)
        return "".join(out)

    return scheme + new_host + repl_path(path)

def marriage_cert_anon(cert: str) -> str:
    # формат NN-NNNNNN
    digits = re.findall(r"\d", cert)
    if len(digits) != 8:
        return "Некорректное свидетельство"
    keep = digits[:2]
    rest = [str(rand.randint(0,9)) for _ in range(6)]
    new_digits = keep + rest

    out, i = [], 0
    for ch in cert:
        if ch.isdigit():
            out.append(new_digits[i]); i += 1
        else:
            out.append(ch)
    return "".join(out)

def cadastral_anon(num: str) -> str:
    # формат XX:XX:XXXXXXXXX:XXXX — сохраним первые два блока как косвенный признак
    digits = re.findall(r"\d", num)
    if len(digits) != 17:
        return "Некорректный кадастровый"
    keep = digits[:4]  # первые 2+2
    rest = [str(rand.randint(0,9)) for _ in range(17-4)]
    new_digits = keep + rest

    out, i = [], 0
    for ch in num:
        if ch.isdigit():
            out.append(new_digits[i]); i += 1
        else:
            out.append(ch)
    return "".join(out)


Writing anon_utils.py


словарь, который ставит в соответствие метке, полученной от нейросети или регулярного выражения, конкретную функцию, которая умеет этот тип данных корректно заменять

In [ ]:
from anon_utils import (
    name_anon, surname_anon, email_anon, inn_anon, phone_anon,
    passport_anon, card_anon, snils_anon, url_anon, marriage_cert_anon, cadastral_anon
)

ANONYMIZERS = {
    "NAME": name_anon,
    "SURNAME": surname_anon,
    "EMAIL": email_anon,
    "INN": inn_anon,
    "PHONE": phone_anon,
    "PASSPORT": passport_anon,
    "CARD": card_anon,
    "SNILS": snils_anon,
    "URL": url_anon,
    "MARRIAGE_CERT": marriage_cert_anon,
    "CADASTRAL": cadastral_anon,
}


превращение текста в обезличенную версию

In [ ]:
class AnonymizationContext:
    def __init__(self):
        self.cache = {}

    def get(self, label, value, fn):
        key = (label, value)
        if key not in self.cache:
            self.cache[key] = fn(value)
        return self.cache[key]


def anonymize_text(text: str) -> str:
    #  извлекаем сущности
    entities = extract_pii(text)

    # нормализуем (убираем частичные и пересекающиеся)
    entities = normalize_spans(text, entities)

    ctx = AnonymizationContext()
    result = text

    #  замена (идем с конца, чтобы не ломать индексы)
    for label, start, end in sorted(entities, key=lambda x: x[1], reverse=True):
        fn = ANONYMIZERS.get(label)
        if not fn:
            continue

        old = result[start:end]
        if label == "NAME":
          new = fn(old, text, start, end)
        else:
          new = ctx.get(label, old, fn)

        result = result[:start] + new + result[end:]

    return result


метод маскирования

In [ ]:
def mask_text_entities(text: str) -> str:
    # 1) извлекаем сущности
    entities = extract_pii(text)

    # 2) нормализуем (убираем мусор и пересечения)
    entities = normalize_spans(text, entities)

    result = text

    # 3) маскируем ТОЛЬКО найденные ПДн
    for label, start, end in sorted(entities, key=lambda x: x[1], reverse=True):
        length = end - start
        result = result[:start] + ("*" * length) + result[end:]

    return result


подготовка данных для расчета метрики BERTScore

In [ ]:
from bert_score import score
import numpy as np

test_texts = [rec.text for rec in records]

MAX_CHARS = 1000

test_texts = [t[:MAX_CHARS] for t in test_texts]

orig_texts = []
anon_texts = []
mask_texts = []

for text in test_texts:
    orig_texts.append(text)
    anon_texts.append(anonymize_text(text))
    mask_texts.append(mask_text_entities(text))



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
from bert_score import score
import numpy as np

P_anon, R_anon, F1_anon = score(
    cands=anon_texts,
    refs=orig_texts,
    lang="ru",
    verbose=True
)

P_mask, R_mask, F1_mask = score(
    cands=mask_texts,
    refs=orig_texts,
    lang="ru",
    verbose=True
)

print("=== BERTScore (F1) ===")
print(f"Синонимическое обезличивание: {F1_anon.mean().item():.4f}")
print(f"Маскирование:               {F1_mask.mean().item():.4f}")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

...

done in 10.56 seconds, 24.05 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/8 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 12.13 seconds, 20.94 sentences/sec
=== BERTScore (F1) ===
Синонимическое обезличивание: 0.9427
Маскирование:               0.8834


In [ ]:
user_text = input("Введите текст: ")

print("\nORIG:", user_text)
print("\nANON:", anonymize_text(user_text))
print("\nMASK:", mask_text_entities(user_text))


Введите текст: 168-853-265 19

ORIG: 168-853-265 19

ANON: 010-880-697 38

MASK: **************


In [ ]:
test_cases = [
    # 1. Базовый тест: ФИО + Паспорт + Телефон
    "Гражданин Сидоров Алексей Петрович, паспорт 4510 123456, телефон 8-903-123-45-67. СНИЛС 168-853-265 19. Свид-во о браке 45-987654.",

    # 2. Тест на контекст (Унисекс имена): Саша (ж) и Саша (м)
    # Здесь программа должна понять род по глаголам "сказала" / "сделал"
    "Саша звонила вчера, она сказала, что пришлет документы.",
    "Саша звонил вчера, он сказал, что сделал отчет.",

    # 3. Тест на валидацию (Контрольные суммы)
    # Первый ИНН (Сбербанка) валидный -> должен замениться.
    # Второй ИНН (набор цифр) невалидный -> должен остаться как есть.
    "Вот настоящий ИНН 500100732259, а вот просто цифры 1234567890 (это не ИНН).",

    # 4. Тест на приоритеты и вложенность
    # Проверка, что имя "ivan" внутри email не заменится отдельно.
    "Напишите мне на почту ivan.petrov@yandex.ru или посетите сайт https://gosuslugi.ru",

    # 5. Тест на падежи (Склонение)
    # Проверка, что PyMorphy сохраняет падеж (Кого? Михаила -> Кого? [Новое Имя])
    "Я передал документы Михаилу, но не видел Дмитрия.",

    # 6. Карта
    "4242 4242 4242 4242, а 4242 4242 4242 4241 (это фейк)"
]


for i, text in enumerate(test_cases, 1):
    print(f"--- ТЕСТОВЫЙ КЕЙС №{i} ---")
    print(f"[ИСХОДНЫЙ]: {text}")

    # Синонимическое обезличивание
    anon = anonymize_text(text)
    print(f"[ANON]:     {anon}")

    # Маскирование (звездочки)
    mask = mask_text_entities(text)
    print(f"[MASK]:     {mask}")
    print()

--- ТЕСТОВЫЙ КЕЙС №1 ---
[ИСХОДНЫЙ]: Гражданин Сидоров Алексей Петрович, паспорт 4510 123456, телефон 8-903-123-45-67. СНИЛС 168-853-265 19. Свид-во о браке 45-987654.
[ANON]:     Гражданин Герасимов Софон Семен, паспорт 4520 256089, телефон 8-903-717-83-91. СНИЛС 219-202-787 52. Свид-во о браке 45-377709.
[MASK]:     Гражданин ******* ******* ********, паспорт ***********, телефон ***************. СНИЛС **************. Свид-во о браке *********.

--- ТЕСТОВЫЙ КЕЙС №2 ---
[ИСХОДНЫЙ]: Саша звонила вчера, она сказала, что пришлет документы.
[ANON]:     Татьяна звонила вчера, она сказала, что пришлет документы.
[MASK]:     **** звонила вчера, она сказала, что пришлет документы.

--- ТЕСТОВЫЙ КЕЙС №3 ---
[ИСХОДНЫЙ]: Саша звонил вчера, он сказал, что сделал отчет.
[ANON]:     Мартьян звонил вчера, он сказал, что сделал отчет.
[MASK]:     **** звонил вчера, он сказал, что сделал отчет.

--- ТЕСТОВЫЙ КЕЙС №4 ---
[ИСХОДНЫЙ]: Вот настоящий ИНН 500100732259, а вот просто цифры 1234567890 (это не